In [2]:
import yaml

import json
import os
import pandas as pd

registry_pth = os.path.join("..", "evals", "registry")
os.makedirs(os.path.join(registry_pth, "data", "calculus"), exist_ok=True)

def create_chat_prompt(prompt, ideal, rubric):
  return dict(
    input = [
      dict(
        role = 'system',
        content = prompt,
      ),
    ],
    ideal = ideal,
    rubric = rubric,
  )

def create_chat_prompt_labeled(prompt, ideal, rubric, completion, choice):
  return dict(
    input = [
      dict(
        role = 'system',
        content = prompt,
      ),
    ],
    ideal = ideal,
    rubric = rubric,
    completion = completion,
    choice = choice,
  )

'c:\\Users\\kaben\\Documents\\GitHub\\irrealis\\openai-evals\\notebooks'

In [64]:
prompt = create_chat_prompt(
  prompt = '''Evaluate the following integral:
$$
\int e^{x} dx
$$''',
  ideal = '''Because the derivative of $e^x$ is $e^x$, it follows that $\int e^x dx$ is simply $e^x + C$.''',
  rubric = '''
- Final Answer Correctness (50 points): The final answer provided by the model matches the expert's answer.
  - 50: The final answer is "e^x + C".
  - 0: The final answer is not "e^x + C".
- Explanation Correctness (50 points): The explanation provided by the model to arrive at the answer is correct and logically sound.
  - 50: The explanation correctly states that the derivative of "e^x" is "e^x", so the integral of "e^x dx" is "e^x + C".
  - 25: The explanation is partially correct but either does not correctly identify the derivative of "e^x" as "e^x" or does not correctly state that the integral of "e^x dx" is "e^x + C".
  - 0: The explanation is incorrect or missing.

There is little nuance in the grading of this problem, as the formula is fundamental and usually memorized as part of a standard integration table.'''
)
prompts = [prompt]
prompts

[{'input': [{'role': 'system',
    'content': 'Evaluate the following integral:\n$$\n\\int e^{x} dx\n$$'}],
  'ideal': 'Because the derivative of $e^x$ is $e^x$, it follows that $\\int e^x dx$ is simply $e^x + C$.',
  'rubric': '\n- Final Answer Correctness (50 points): The final answer provided by the model matches the expert\'s answer.\n  - 50: The final answer is "e^x + C".\n  - 0: The final answer is not "e^x + C".\n- Explanation Correctness (50 points): The explanation provided by the model to arrive at the answer is correct and logically sound.\n  - 50: The explanation correctly states that the derivative of "e^x" is "e^x", so the integral of "e^x dx" is "e^x + C".\n  - 25: The explanation is partially correct but either does not correctly identify the derivative of "e^x" as "e^x" or does not correctly state that the integral of "e^x dx" is "e^x + C".\n  - 0: The explanation is incorrect or missing.\n\nThere is little nuance in the grading of this problem, as the formula is funda

In [67]:
df = pd.DataFrame(
  data = dict(
    input = [p['input'] for p in prompts],
    ideal = [p['ideal'] for p in prompts],
    rubric = [p['rubric'] for p in prompts],
  )
)
df

,input,ideal,rubric
0,"[{'role': 'system', 'content': 'Evaluate the f...","Because the derivative of $e^x$ is $e^x$, it f...",\n- Final Answer Correctness (50 points): The ...


In [71]:
demo_pth = os.path.join(registry_pth, 'data', 'calculus', 'demo.jsonl')
os.remove(demo_pth)
df[['input', 'ideal', 'rubric']].to_json(demo_pth, lines=True, orient="records")

In [74]:
logfiles = os.listdir('/tmp/evallogs/')
logfiles.sort()
logfiles

['230520222738MXRJLY3S_gpt-3.5-turbo_test-match.jsonl',
 '230520223055VONH5IV3_gpt-3.5-turbo_test-match.jsonl',
 '230520223256KETQMTZK_gpt-3.5-turbo_test-match.jsonl',
 '230521050221DK7GBI5E_gpt-3.5-turbo_lambada.jsonl',
 '2305210630073JJ2YZX3_gpt-3.5-turbo_test-match.jsonl',
 '230521063016GCLA67AX_gpt-3.5-turbo_calculus.jsonl',
 '230521064003ZX6CMR3C_gpt-3.5-turbo_calculus.jsonl',
 '230521065918QNDMUMW3_gpt-3.5-turbo_calculus.jsonl',
 '230521070112BCUEO4NZ_gpt-3.5-turbo_calculus.jsonl',
 '230521070826AALQ3GPC_gpt-3.5-turbo_calculus.jsonl',
 '230521071107CFD45PFO_gpt-3.5-turbo_calculus.jsonl',
 '2305210712373AURJBGC_gpt-3.5-turbo_calculus.jsonl',
 '230521234127DQBEUDEL_gpt-3.5-turbo_calculus.jsonl']

In [72]:
!oaieval gpt-3.5-turbo calculus

[2023-05-21 17:41:27,640] [registry.py:249] Loading registry from C:\Users\kaben\Documents\GitHub\irrealis\openai-evals\evals\registry\evals
[2023-05-21 17:41:27,938] [registry.py:249] Loading registry from C:\Users\kaben\.evals\evals
[2023-05-21 17:41:27,940] [oaieval.py:110] Run started: 230521234127DQBEUDEL
[2023-05-21 17:41:27,941] [registry.py:249] Loading registry from C:\Users\kaben\Documents\GitHub\irrealis\openai-evals\evals\registry\modelgraded
[2023-05-21 17:41:27,964] [registry.py:249] Loading registry from C:\Users\kaben\.evals\modelgraded
[2023-05-21 17:41:27,964] [data.py:75] Fetching calculus/demo.jsonl
[2023-05-21 17:41:27,968] [eval.py:33] Evaluating 1 samples
[2023-05-21 17:41:27,972] [eval.py:138] Running in threaded mode with 10 threads!

100%|██████████| 1/1 [00:16<00:00, 16.80s/it]
[2023-05-21 17:41:44,771] [record.py:341] Final report: {'counts/0': 1, 'score': 0.0}. Logged to /tmp/evallogs/230521234127DQBEUDEL_gpt-3.5-turbo_calculus.jsonl
[2023-05-21 17:41:44,77

In [1]:
# How to process the log events generated by oaieval
#log_name = '230521064003ZX6CMR3C_gpt-3.5-turbo_calculus.jsonl'
#log_name = '230521065918QNDMUMW3_gpt-3.5-turbo_calculus.jsonl
#log_name = '230521070112BCUEO4NZ_gpt-3.5-turbo_calculus.jsonl'
#log_name = '230521070826AALQ3GPC_gpt-3.5-turbo_calculus.jsonl'
#log_name = '230521071107CFD45PFO_gpt-3.5-turbo_calculus.jsonl'
#log_name = '2305210712373AURJBGC_gpt-3.5-turbo_calculus.jsonl'

logfiles = os.listdir('/tmp/evallogs/')
logfiles.sort()
print(logfiles)
log_name = logfiles[-1]
print(log_name)
events = f"/tmp/evallogs/{log_name}"
with open(events, "r") as f:
    events_df = pd.read_json(f, lines=True)

NameError: name 'os' is not defined

In [93]:
for entry in events_df['data']:
    if entry == entry:
        if 'prompt' in entry:
            print('*' * 40, 'prompt')
            for x in entry['prompt']:
                print(x['content'])
        if 'sampled' in entry:
            print('-' * 40, 'sampled')
            for x in entry['sampled']:
                print(x)

**************************************** prompt
Evaluate the following integral:
$$
\int e^{x} dx
$$
---------------------------------------- sampled
We know that the derivative of $e^x$ is itself, so we can use integration by substitution. Let $u = x$ and $du = dx$. Then we have:
$$
\int e^{x} dx = \int e^{u} du = e^u + C = e^x + C
$$
where $C$ is the constant of integration. Therefore, the solution to the integral is $\boxed{e^x + C}$.
**************************************** prompt
You are comparing a submitted answer to an expert answer for a given mathematics problem:
---
[Mathematics Problem]: Evaluate the following integral:
$$
\int e^{{x}} dx
$$
---
[Expert]: Because the derivative of $e^x$ is $e^x$, it follows that $\int e^x dx$ is simply $e^x + C$.
---
[Submission]: We know that the derivative of $e^x$ is itself, so we can use integration by substitution. Let $u = x$ and $du = dx$. Then we have:
$$
\int e^{x} dx = \int e^{u} du = e^u + C = e^x + C
$$
where $C$ is the constant

In [94]:
events_df

,spec,final_report,run_id,event_id,sample_id,type,data,created_by,created_at
0,"{'completion_fns': ['gpt-3.5-turbo'], 'eval_na...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,NaN,"{'counts/100': 1, 'score': 1.0}",NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,NaN,NaN,230521235705QVVVZTWJ,0.0,calculus.dev.0,sampling,"{'prompt': [{'role': 'system', 'content': 'Eva...",,2023-05-21 23:57:11.902654+00:00
3,NaN,NaN,230521235705QVVVZTWJ,1.0,calculus.dev.0,sampling,"{'prompt': [{'role': 'user', 'content': 'You a...",,2023-05-21 23:57:15.977631+00:00
4,NaN,NaN,230521235705QVVVZTWJ,2.0,calculus.dev.0,metrics,"{'choice': '100', 'score': 1.0}",,2023-05-21 23:57:16.005656+00:00


In [121]:
print(events_df['data'][3]['prompt'][0]['content'])

You are comparing a submitted answer to an expert answer for a given mathematics problem:
---
[Mathematics Problem]: Evaluate the following integral:
$$
\int e^{{x}} dx
$$
---
[Expert]: Because the derivative of $e^x$ is $e^x$, it follows that $\int e^x dx$ is simply $e^x + C$.
---
[Submission]: We know that the derivative of $e^x$ is itself, so we can use integration by substitution. Let $u = x$ and $du = dx$. Then we have:
$$
\int e^{x} dx = \int e^{u} du = e^u + C = e^x + C
$$
where $C$ is the constant of integration. Therefore, the solution to the integral is $\boxed{e^x + C}$.
---
Use the following rubric to score the submission. Round the score to the nearest integer in the range 0-100 inclusive.

[Rubric]: 
- Final Answer Correctness (50 points): The final answer provided by the model matches the expert's answer.
  - 50: The final answer is "e^x + C".
  - 0: The final answer is not "e^x + C".
- Explanation Correctness (50 points): The explanation provided by the model to arrive 

In [102]:
print(events_df.loc(3)['data']['prompt']['content'])


ValueError: No axis named 3 for object type DataFrame

In [77]:
print({"run_id": "230521070112BCUEO4NZ", "event_id": 0, "sample_id": "calculus.dev.0", "type": "sampling", "data": {"prompt": [{"role": "system", "content": "Evaluate the following integral:\n$$\n\\int e^{x} dx\n$$"}], "sampled": ["We know that the derivative of $e^x$ is itself, so we can use integration by substitution. Let $u = x$ and $du = dx$. Then we have:\n$$\n\\int e^{x} dx = \\int e^{u} du = e^u + C = e^x + C\n$$\nwhere $C$ is the constant of integration. Therefore, the solution to the integral is $\\boxed{e^x + C}$."]}, "created_by": "", "created_at": "2023-05-21 07:01:21.570168+00:00"})
print({"run_id": "230521070112BCUEO4NZ", "event_id": 1, "sample_id": "calculus.dev.0", "type": "sampling", "data": {"prompt": [{"role": "user", "content": "You are comparing a submitted answer to an expert answer for a given mathematics problem:\n***\n[Mathematics Problem]: Evaluate the following integral:\n$$\n\\int e^{{x}} dx\n$$\n***\n[Expert]: Because the derivative of $e^x$ is $e^x$, it follows that $\\int e^x dx$ is simply $e^x + C$.\n***\n[Submission]: We know that the derivative of $e^x$ is itself, so we can use integration by substitution. Let $u = x$ and $du = dx$. Then we have:\n$$\n\\int e^{x} dx = \\int e^{u} du = e^u + C = e^x + C\n$$\nwhere $C$ is the constant of integration. Therefore, the solution to the integral is $\\boxed{e^x + C}$.\n***\nIs the submitted answer equivalent to the expert answer? Take this step by step to make sure that your conclusion is correct. Avoid simply stating the correct answer at the outset. Then print only the single character \"Y\" or \"N\" (without quotes or punctuation) on its own line corresponding to the correct answer. At the end, repeat just the letter again by itself on a new line.\n\nReasoning:\n\nFirst, write out in a step by step manner your reasoning to be sure that your conclusion is correct. Avoid simply stating the correct answer at the outset. Then print only a single choice from \"Y\" or \"N\" (without quotes or punctuation) on its own line corresponding to the correct answer. At the end, repeat just the answer by itself on a new line.\n\nReasoning:"}], "sampled": ["The submitted answer and the expert answer both arrive at the same solution of $e^x + C$. The submitted answer uses integration by substitution to arrive at the antiderivative, while the expert answer uses the fact that the derivative of $e^x$ is itself. However, both methods are valid and lead to the same result. Therefore, the submitted answer is equivalent to the expert answer. \n\nY"]}, "created_by": "", "created_at": "2023-05-21 07:01:29.354595+00:00"})
print({"run_id": "230521070112BCUEO4NZ", "event_id": 2, "sample_id": "calculus.dev.0", "type": "metrics", "data": {"choice": "Y", "score": 1.0}, "created_by": "", "created_at": "2023-05-21 07:01:29.354595+00:00"})

{'run_id': '230521070112BCUEO4NZ', 'event_id': 0, 'sample_id': 'calculus.dev.0', 'type': 'sampling', 'data': {'prompt': [{'role': 'system', 'content': 'Evaluate the following integral:\n$$\n\\int e^{x} dx\n$$'}], 'sampled': ['We know that the derivative of $e^x$ is itself, so we can use integration by substitution. Let $u = x$ and $du = dx$. Then we have:\n$$\n\\int e^{x} dx = \\int e^{u} du = e^u + C = e^x + C\n$$\nwhere $C$ is the constant of integration. Therefore, the solution to the integral is $\\boxed{e^x + C}$.']}, 'created_by': '', 'created_at': '2023-05-21 07:01:21.570168+00:00'}
{'run_id': '230521070112BCUEO4NZ', 'event_id': 1, 'sample_id': 'calculus.dev.0', 'type': 'sampling', 'data': {'prompt': [{'role': 'user', 'content': 'You are comparing a submitted answer to an expert answer for a given mathematics problem:\n***\n[Mathematics Problem]: Evaluate the following integral:\n$$\n\\int e^{{x}} dx\n$$\n***\n[Expert]: Because the derivative of $e^x$ is $e^x$, it follows that $

In [53]:
import json

In [80]:
print(json.dumps({f'{i}': i/100 for i in range(100, -1, -1)}))

{"100": 1.0, "99": 0.99, "98": 0.98, "97": 0.97, "96": 0.96, "95": 0.95, "94": 0.94, "93": 0.93, "92": 0.92, "91": 0.91, "90": 0.9, "89": 0.89, "88": 0.88, "87": 0.87, "86": 0.86, "85": 0.85, "84": 0.84, "83": 0.83, "82": 0.82, "81": 0.81, "80": 0.8, "79": 0.79, "78": 0.78, "77": 0.77, "76": 0.76, "75": 0.75, "74": 0.74, "73": 0.73, "72": 0.72, "71": 0.71, "70": 0.7, "69": 0.69, "68": 0.68, "67": 0.67, "66": 0.66, "65": 0.65, "64": 0.64, "63": 0.63, "62": 0.62, "61": 0.61, "60": 0.6, "59": 0.59, "58": 0.58, "57": 0.57, "56": 0.56, "55": 0.55, "54": 0.54, "53": 0.53, "52": 0.52, "51": 0.51, "50": 0.5, "49": 0.49, "48": 0.48, "47": 0.47, "46": 0.46, "45": 0.45, "44": 0.44, "43": 0.43, "42": 0.42, "41": 0.41, "40": 0.4, "39": 0.39, "38": 0.38, "37": 0.37, "36": 0.36, "35": 0.35, "34": 0.34, "33": 0.33, "32": 0.32, "31": 0.31, "30": 0.3, "29": 0.29, "28": 0.28, "27": 0.27, "26": 0.26, "25": 0.25, "24": 0.24, "23": 0.23, "22": 0.22, "21": 0.21, "20": 0.2, "19": 0.19, "18": 0.18, "17": 0.17,

In [81]:
print(json.dumps([f'{i}' for i in range(100, -1, -1)]))

["100", "99", "98", "97", "96", "95", "94", "93", "92", "91", "90", "89", "88", "87", "86", "85", "84", "83", "82", "81", "80", "79", "78", "77", "76", "75", "74", "73", "72", "71", "70", "69", "68", "67", "66", "65", "64", "63", "62", "61", "60", "59", "58", "57", "56", "55", "54", "53", "52", "51", "50", "49", "48", "47", "46", "45", "44", "43", "42", "41", "40", "39", "38", "37", "36", "35", "34", "33", "32", "31", "30", "29", "28", "27", "26", "25", "24", "23", "22", "21", "20", "19", "18", "17", "16", "15", "14", "13", "12", "11", "10", "9", "8", "7", "6", "5", "4", "3", "2", "1", "0"]
